In [1]:
# setting the random seed for reproducibility
import random
random.seed(493)

# for manipulating dataframes
import pandas as pd
import numpy as np

# for statistical testing
from scipy import stats

# for working with timestamps
from datetime import datetime
from dateutil.parser import parse

# for visualizations
%matplotlib inline
import matplotlib.pyplot as plt

# to print out all the outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

# Executive Summary

> **Something goes here.**

### PLANNING

- [X] Planning
- [X] Acquire data
- [ ] Prepare the data
- [ ] Explore the data
- [ ] Modeling

In [2]:
def show_missing(df):
    """
    Takes a dataframe and returns a dataframe with stats
    on missing and null values with their percentages.
    """
    null_count = df.isnull().sum()
    null_percentage = (null_count / df.shape[0]) * 100
    empty_count = pd.Series(((df == ' ') | (df == '')).sum())
    empty_percentage = (empty_count / df.shape[0]) * 100
    nan_count = pd.Series(((df == 'nan') | (df == 'NaN')).sum())
    nan_percentage = (nan_count / df.shape[0]) * 100
    dfx = pd.DataFrame({'num_missing': null_count, 'missing_percentage': null_percentage,
                         'num_empty': empty_count, 'empty_percentage': empty_percentage,
                         'nan_count': nan_count, 'nan_percentage': nan_percentage})
    return dfx

In [3]:
def get_values(df, columns):
    """
    Take a dataframe and a list of columns and
    returns the value counts for the columns.
    """
    for column in columns:
        print(column)
        print('=====================================')
        print(df[column].value_counts(dropna=False))
        print('\n')

def show_values(df, param):
    if param == 'all':
        get_values(df, df.columns)
    else:
        get_values(df, param)

In [4]:
def convert_datatype(df, column, totype):
    df[column] = df[column].astype(totype)
    return df

### ACQUIRE DATA

In [5]:
# Read csv files
peims_df = pd.read_csv('../data/in/2007-2021-summaried-peims-financial-data.csv')

In [6]:
peims_df.head()

,DISTRICT NUMBER,DISTRICT NAME,YEAR,GEN FUNDS-LOCAL TAX REVENUE FROM M&O,ALL FUNDS-LLOCAL TAX REVENUE FROM M&O,GEN FUNDS-STATE REVENUE,ALL FUNDS-STATE REVENUE,GEN FUNDS-FEDERAL REVENUE,ALL FUNDS-FEDERAL REVENUE,GEN FUNDS-OTHER LOCAL & INTERMEDIATE REVENUE,ALL FUNDS-OTHER LOCAL & INTERMEDIATE REVENUE,GEN FUNDS-TOTAL OPERATING REVENUE,ALL FUNDS-TOTAL OPERATING REVENUE,GEN FUNDS-LOCAL PROPERTY TAXES FROM I&S,ALL FUNDS-LOCAL PROPERTY TAXES FROM I&S,GEN FUNDS-STATE DEBT FUNDS,ALL FUNDS-STATE DEBT FUNDS,GEN FUNDS-OTHER RECEIPTS,ALL FUNDS-OTHER RECEIPTS,GEN FUNDS-OTHER REVENUE,ALL FUNDS-OTHER REVENUE,GEN FUNDS-TOTAL OPERATING REVENUE AND OTHER REVENUE,ALL FUNDS-TOTAL OPERATING REVENUE AND OTHER REVENUE,GEN FUNDS-EQUITY TRANSFERS,ALL FUNDS-EQUITY TRANSFERS,GEN FUNDS-TOT DEBT SERV FIN AND TRS EST REV,ALL FUNDS-TOT DEBT SERV FIN AND TRS EST REV,GEN FUNDS-TOTAL OPERATING REVENUE AND OTHER REVENUE AND RECPATUR,ALL FUNDS-TOTAL OPERATING REVENUE AND OTHER REVENUE AND RECPATUR,GEN FUNDS-DEBT SERVICE FINANCING RELATED REVENUE,ALL FUNDS-DEBT SERVICE FINANCING RELATED REVENUE,GEN FUNDS-ESTIMATED STATE TRS CONTRINUTIONS,ALL FUNDS-ESTIMATED STATE TRS CONTRINUTIONS,GEN FUNDS-TOTAL DEBT SERVICE FINANCING AND TRS ESTIMATE REVENUE,ALL FUNDS-TOTAL DEBT SERVICE FINANCING AND TRS ESTIMATE REVENUE,"GEN FUNDS-TOTAL OPERATING, OTR, DEBT SERV FIN, AND TRS EST REVEN","ALL FUNDS-TOTAL OPERATING, OTR, DEBT SERV FIN, AND TRS EST REVEN",GEN FUNDS-TOTAL PAYROLL EXPENDITURES,ALL FUNDS-TOTAL PAYROLL EXPENDITURES,GEN FUNDS-TOTAL PROFESSIONAL & CONTRACTED SERVICES EXPENDITURES,ALL FUNDS-TOTAL PROFESSIONAL & CONTRACTED SERVICES EXPENDITURES,GEN FUNDS-TOTAL SUPPLIES & MATERIALS EXPENDITURES,ALL FUNDS-TOTAL SUPPLIES & MATERIALS EXPENDITURES,GEN FUNDS-TOTAL OTHER OPERATING EXPENDITURES,ALL FUNDS-TOTAL OTHER OPERATING EXPENDITURES,GEN FUNDS-TOTAL OPERATING EXPENDITURES BY OBJ,ALL FUNDS-TOTAL OPERATING EXPENDITURES BY OBJ,GEN FUNDS-TOTAL DEBT SERVICE EXPEND BY OBJ,ALL FUNDS-TOTAL DEBT SERVICE EXPEND BY OBJ,GEN FUNDS-TOTAL CAPITAL OUTLAY EXPEND BY OBJ,ALL FUNDS-TOTAL CAPITAL OUTLAY EXPEND BY OBJ,GEN FUNDS-TOTAL NON-OPER EXPENDITURES BY OBJ,ALL FUNDS-TOTAL NON-OPER EXPENDITURES BY OBJ,GEN FUNDS-TOTAL NON-OPER AND OPER OEXPENDITURES BY OBJ,ALL FUNDS-TOTAL NON-OPER AND OPER OEXPENDITURES BY OBJ,"GEN FUNDS-INSTRUCTION + TRANSFER EXPEND-FCT11,95","ALL FUNDS-INSTRUCTION + TRANSFER EXPEND-FCT11,95","GEN FUNDS-INSTRUC RESOURCE MEDIA SERVICE EXP, FCT12","ALL FUNDS-INSTRUC RESOURCE MEDIA SERVICE, FCT12","GEN FUNDS-CURRICULUM/STAFF DEVELOPMENT EXP, FCT13","ALL FUNDS-CURRICULUM/STAFF DEVELOPMENT EXP, FCT13","GEN FUNDS-INSTRUC LEADERSHIP EXPEND, FCT21","ALL FUNDS-INSTRUC LEADERSHIP EXPEND, FCT21","GEN FUNDS-CAMPUS ADMINISTRATION EXPEND, FCT23","ALL FUNDS-CAMPUS ADMINISTRATION EXPEND, FCT23","GEN FUNDS-GUIDANCE & COUNSELING SERVICES EXP, FCT31","ALL FUNDS-GUIDANCE 7 COUNSELING SERVICES EXP, FCT31","GEN FUNDS-SOCIAL WORK SERVICES EXP, FCT32","ALL FUNDS-SOCIAL WORK SERVICES EXP, FCT32","GEN FUNDS-HEALTH SERVICES EXP, FCT33","ALL FUNDS-HEALTH SERVICES EXP, FCT33","GEN FUNDS-TRANSPORTATION EXPENDITURES, FCT34","ALL FUNDS-TRANSPORTATION EXPENDITURES, FCT34","GEN FUNDS-FOOD SERVICE EXPENDITURES, FCT35","ALL FUNDS-FOOD SERVICE EXPENDITURES, FCT35","GEN FUNDS-EXTRACURRICULAR EXPENDITURES, FCT36","ALL FUNDS-EXTRACURRICULAR EXPENDITURES,FCT36","GEN FUNDS-GENERAL ADMINISTRAT EXPEND-FCT41,80,92","ALL FUNDS-GENERAL ADMINISTRAT EXPEND-FCT41,80,92","GEN FUNDS-PLANT MAINTENANCE/OPERA EXPEND, FCT51","ALL FUNDS-PLANT MAINTENANCE/OPERA EXPEND, FCT51","GEN FUNDS-SECURITY/MONITORING SERVICE EXPEND, FCT52","ALL FUNDS-SECURITY/MONITORING SERVICE EXPEND, FCT5","GEN FUNDS-DATA PROCESSING SERVICES EXPEND, FCT53","ALL FUNDS-DATA PROCESSING SERVICES, FCT53","GEN FUNDS-COMMUNITY SERVICES, FCT61","ALL FUNDS-COMMUNITY SERVICES, FCT61",GEN FUNDS-TOTAL OPERATE EXPEND BY FUNCTION,ALL FUNDS-TOTAL OPERATE EXPEND BY FUNCTION,GEN FUNDS-NON-OPER EXP BY FUNCTION(1X-9X)(65XX),ALL FUNDS-NON-OPER EXP BY FUNCTION(1X-9X)(65XX),GEN 

### PREPARATION

In [7]:
peims_df.shape

(18213, 150)

In [8]:
peims_df.head().T

,0,1,2,3,4
DISTRICT NUMBER,'001902,'001902,'001902,'001902,'001902
DISTRICT NAME,CAYUGA ISD,CAYUGA ISD,CAYUGA ISD,CAYUGA ISD,CAYUGA ISD
YEAR,2007,2008,2009,2010,2011
GEN FUNDS-LOCAL TAX REVENUE FROM M&O,4122552,3641351,3319164,3222688,3152618
ALL FUNDS-LLOCAL TAX REVENUE FROM M&O,4122552,3641351,3319164,3222688,3152618
GEN FUNDS-STATE REVENUE,1114179,1845232,1950169,2302369,2439570
ALL FUNDS-STATE REVENUE,1176283,2350131,2473373,2917268,3091438
GEN FUNDS-FEDERAL REVENUE,0,0,0,0,0
ALL FUNDS-FEDERAL REVENUE,250168,1852221,1714988,2851372,2687377
GEN FUNDS-OTHER LOCAL & INTERMEDIATE REVENUE,215239,183293,123568,161664,134887


In [9]:
peims_df.head()

,DISTRICT NUMBER,DISTRICT NAME,YEAR,GEN FUNDS-LOCAL TAX REVENUE FROM M&O,ALL FUNDS-LLOCAL TAX REVENUE FROM M&O,GEN FUNDS-STATE REVENUE,ALL FUNDS-STATE REVENUE,GEN FUNDS-FEDERAL REVENUE,ALL FUNDS-FEDERAL REVENUE,GEN FUNDS-OTHER LOCAL & INTERMEDIATE REVENUE,ALL FUNDS-OTHER LOCAL & INTERMEDIATE REVENUE,GEN FUNDS-TOTAL OPERATING REVENUE,ALL FUNDS-TOTAL OPERATING REVENUE,GEN FUNDS-LOCAL PROPERTY TAXES FROM I&S,ALL FUNDS-LOCAL PROPERTY TAXES FROM I&S,GEN FUNDS-STATE DEBT FUNDS,ALL FUNDS-STATE DEBT FUNDS,GEN FUNDS-OTHER RECEIPTS,ALL FUNDS-OTHER RECEIPTS,GEN FUNDS-OTHER REVENUE,ALL FUNDS-OTHER REVENUE,GEN FUNDS-TOTAL OPERATING REVENUE AND OTHER REVENUE,ALL FUNDS-TOTAL OPERATING REVENUE AND OTHER REVENUE,GEN FUNDS-EQUITY TRANSFERS,ALL FUNDS-EQUITY TRANSFERS,GEN FUNDS-TOT DEBT SERV FIN AND TRS EST REV,ALL FUNDS-TOT DEBT SERV FIN AND TRS EST REV,GEN FUNDS-TOTAL OPERATING REVENUE AND OTHER REVENUE AND RECPATUR,ALL FUNDS-TOTAL OPERATING REVENUE AND OTHER REVENUE AND RECPATUR,GEN FUNDS-DEBT SERVICE FINANCING RELATED REVENUE,ALL FUNDS-DEBT SERVICE FINANCING RELATED REVENUE,GEN FUNDS-ESTIMATED STATE TRS CONTRINUTIONS,ALL FUNDS-ESTIMATED STATE TRS CONTRINUTIONS,GEN FUNDS-TOTAL DEBT SERVICE FINANCING AND TRS ESTIMATE REVENUE,ALL FUNDS-TOTAL DEBT SERVICE FINANCING AND TRS ESTIMATE REVENUE,"GEN FUNDS-TOTAL OPERATING, OTR, DEBT SERV FIN, AND TRS EST REVEN","ALL FUNDS-TOTAL OPERATING, OTR, DEBT SERV FIN, AND TRS EST REVEN",GEN FUNDS-TOTAL PAYROLL EXPENDITURES,ALL FUNDS-TOTAL PAYROLL EXPENDITURES,GEN FUNDS-TOTAL PROFESSIONAL & CONTRACTED SERVICES EXPENDITURES,ALL FUNDS-TOTAL PROFESSIONAL & CONTRACTED SERVICES EXPENDITURES,GEN FUNDS-TOTAL SUPPLIES & MATERIALS EXPENDITURES,ALL FUNDS-TOTAL SUPPLIES & MATERIALS EXPENDITURES,GEN FUNDS-TOTAL OTHER OPERATING EXPENDITURES,ALL FUNDS-TOTAL OTHER OPERATING EXPENDITURES,GEN FUNDS-TOTAL OPERATING EXPENDITURES BY OBJ,ALL FUNDS-TOTAL OPERATING EXPENDITURES BY OBJ,GEN FUNDS-TOTAL DEBT SERVICE EXPEND BY OBJ,ALL FUNDS-TOTAL DEBT SERVICE EXPEND BY OBJ,GEN FUNDS-TOTAL CAPITAL OUTLAY EXPEND BY OBJ,ALL FUNDS-TOTAL CAPITAL OUTLAY EXPEND BY OBJ,GEN FUNDS-TOTAL NON-OPER EXPENDITURES BY OBJ,ALL FUNDS-TOTAL NON-OPER EXPENDITURES BY OBJ,GEN FUNDS-TOTAL NON-OPER AND OPER OEXPENDITURES BY OBJ,ALL FUNDS-TOTAL NON-OPER AND OPER OEXPENDITURES BY OBJ,"GEN FUNDS-INSTRUCTION + TRANSFER EXPEND-FCT11,95","ALL FUNDS-INSTRUCTION + TRANSFER EXPEND-FCT11,95","GEN FUNDS-INSTRUC RESOURCE MEDIA SERVICE EXP, FCT12","ALL FUNDS-INSTRUC RESOURCE MEDIA SERVICE, FCT12","GEN FUNDS-CURRICULUM/STAFF DEVELOPMENT EXP, FCT13","ALL FUNDS-CURRICULUM/STAFF DEVELOPMENT EXP, FCT13","GEN FUNDS-INSTRUC LEADERSHIP EXPEND, FCT21","ALL FUNDS-INSTRUC LEADERSHIP EXPEND, FCT21","GEN FUNDS-CAMPUS ADMINISTRATION EXPEND, FCT23","ALL FUNDS-CAMPUS ADMINISTRATION EXPEND, FCT23","GEN FUNDS-GUIDANCE & COUNSELING SERVICES EXP, FCT31","ALL FUNDS-GUIDANCE 7 COUNSELING SERVICES EXP, FCT31","GEN FUNDS-SOCIAL WORK SERVICES EXP, FCT32","ALL FUNDS-SOCIAL WORK SERVICES EXP, FCT32","GEN FUNDS-HEALTH SERVICES EXP, FCT33","ALL FUNDS-HEALTH SERVICES EXP, FCT33","GEN FUNDS-TRANSPORTATION EXPENDITURES, FCT34","ALL FUNDS-TRANSPORTATION EXPENDITURES, FCT34","GEN FUNDS-FOOD SERVICE EXPENDITURES, FCT35","ALL FUNDS-FOOD SERVICE EXPENDITURES, FCT35","GEN FUNDS-EXTRACURRICULAR EXPENDITURES, FCT36","ALL FUNDS-EXTRACURRICULAR EXPENDITURES,FCT36","GEN FUNDS-GENERAL ADMINISTRAT EXPEND-FCT41,80,92","ALL FUNDS-GENERAL ADMINISTRAT EXPEND-FCT41,80,92","GEN FUNDS-PLANT MAINTENANCE/OPERA EXPEND, FCT51","ALL FUNDS-PLANT MAINTENANCE/OPERA EXPEND, FCT51","GEN FUNDS-SECURITY/MONITORING SERVICE EXPEND, FCT52","ALL FUNDS-SECURITY/MONITORING SERVICE EXPEND, FCT5","GEN FUNDS-DATA PROCESSING SERVICES EXPEND, FCT53","ALL FUNDS-DATA PROCESSING SERVICES, FCT53","GEN FUNDS-COMMUNITY SERVICES, FCT61","ALL FUNDS-COMMUNITY SERVICES, FCT61",GEN FUNDS-TOTAL OPERATE EXPEND BY FUNCTION,ALL FUNDS-TOTAL OPERATE EXPEND BY FUNCTION,GEN FUNDS-NON-OPER EXP BY FUNCTION(1X-9X)(65XX),ALL FUNDS-NON-OPER EXP BY FUNCTION(1X-9X)(65XX),GEN 

In [10]:
peims_df.columns

Index(['DISTRICT NUMBER', 'DISTRICT NAME', 'YEAR',
       'GEN FUNDS-LOCAL TAX REVENUE FROM M&O',
       'ALL FUNDS-LLOCAL TAX REVENUE FROM M&O', 'GEN FUNDS-STATE REVENUE',
       'ALL FUNDS-STATE REVENUE', 'GEN FUNDS-FEDERAL REVENUE',
       'ALL FUNDS-FEDERAL REVENUE',
       'GEN FUNDS-OTHER LOCAL & INTERMEDIATE REVENUE',
       ...
       'ALL FUNDS-TOTAL OTHER USES', 'GEN FUNDS-INTERGOVERN CHARGES EXPEND',
       'ALL FUNDS-INTERGOVERN CHARGES EXPEND',
       'GEN FUNDS-DEBT SERVICE (OBJECT 6500) FOR TD',
       'ALL FUNDS-DEBT SERVICE (OBJECT 6500) FOR TD',
       'GEN FUNDS-CAPITAL PROJECTS(OBJECT 6600) FOR TD',
       'ALL FUNDS-CAPITAL PROJECTS(OBJECT 6600) FOR TD',
       'GEN FUNDS-TOTAL DISBURSEMENTS', 'ALL FUNDS-TOTAL DISBURSEMENTS',
       'FALL SURVEY ENROLLMENT'],
      dtype='object', length=150)

In [11]:
show_values(peims_df, ['DISTRICT NUMBER', 'YEAR'])

DISTRICT NUMBER
'001902    15
'152910    15
'155901    15
'154903    15
'154901    15
'153907    15
'153905    15
'153904    15
'153903    15
'152909    15
'150901    15
'152908    15
'152907    15
'152906    15
'152903    15
'152902    15
'152901    15
'152803    15
'156902    15
'156905    15
'157901    15
'158901    15
'161908    15
'161907    15
'161906    15
'161903    15
'161901    15
'161802    15
'161801    15
'160905    15
'160904    15
'160901    15
'159901    15
'158906    15
'158905    15
'158904    15
'158902    15
'152802    15
'149902    15
'161910    15
'143901    15
'144902    15
'144901    15
'143906    15
'143905    15
'143904    15
'143903    15
'143902    15
'142901    15
'149901    15
'141902    15
'141901    15
'140908    15
'140907    15
'140905    15
'140904    15
'140901    15
'144903    15
'145901    15
'145902    15
'145906    15
'148905    15
'148902    15
'148901    15
'147903    15
'147902    15
'147901    15
'146907    15
'146906    15
'146905    15
'146

In [12]:
peims_df['DISTRICT NUMBER'] = peims_df['DISTRICT NUMBER'].str[1:]

In [13]:
peims_df.head()

,DISTRICT NUMBER,DISTRICT NAME,YEAR,GEN FUNDS-LOCAL TAX REVENUE FROM M&O,ALL FUNDS-LLOCAL TAX REVENUE FROM M&O,GEN FUNDS-STATE REVENUE,ALL FUNDS-STATE REVENUE,GEN FUNDS-FEDERAL REVENUE,ALL FUNDS-FEDERAL REVENUE,GEN FUNDS-OTHER LOCAL & INTERMEDIATE REVENUE,ALL FUNDS-OTHER LOCAL & INTERMEDIATE REVENUE,GEN FUNDS-TOTAL OPERATING REVENUE,ALL FUNDS-TOTAL OPERATING REVENUE,GEN FUNDS-LOCAL PROPERTY TAXES FROM I&S,ALL FUNDS-LOCAL PROPERTY TAXES FROM I&S,GEN FUNDS-STATE DEBT FUNDS,ALL FUNDS-STATE DEBT FUNDS,GEN FUNDS-OTHER RECEIPTS,ALL FUNDS-OTHER RECEIPTS,GEN FUNDS-OTHER REVENUE,ALL FUNDS-OTHER REVENUE,GEN FUNDS-TOTAL OPERATING REVENUE AND OTHER REVENUE,ALL FUNDS-TOTAL OPERATING REVENUE AND OTHER REVENUE,GEN FUNDS-EQUITY TRANSFERS,ALL FUNDS-EQUITY TRANSFERS,GEN FUNDS-TOT DEBT SERV FIN AND TRS EST REV,ALL FUNDS-TOT DEBT SERV FIN AND TRS EST REV,GEN FUNDS-TOTAL OPERATING REVENUE AND OTHER REVENUE AND RECPATUR,ALL FUNDS-TOTAL OPERATING REVENUE AND OTHER REVENUE AND RECPATUR,GEN FUNDS-DEBT SERVICE FINANCING RELATED REVENUE,ALL FUNDS-DEBT SERVICE FINANCING RELATED REVENUE,GEN FUNDS-ESTIMATED STATE TRS CONTRINUTIONS,ALL FUNDS-ESTIMATED STATE TRS CONTRINUTIONS,GEN FUNDS-TOTAL DEBT SERVICE FINANCING AND TRS ESTIMATE REVENUE,ALL FUNDS-TOTAL DEBT SERVICE FINANCING AND TRS ESTIMATE REVENUE,"GEN FUNDS-TOTAL OPERATING, OTR, DEBT SERV FIN, AND TRS EST REVEN","ALL FUNDS-TOTAL OPERATING, OTR, DEBT SERV FIN, AND TRS EST REVEN",GEN FUNDS-TOTAL PAYROLL EXPENDITURES,ALL FUNDS-TOTAL PAYROLL EXPENDITURES,GEN FUNDS-TOTAL PROFESSIONAL & CONTRACTED SERVICES EXPENDITURES,ALL FUNDS-TOTAL PROFESSIONAL & CONTRACTED SERVICES EXPENDITURES,GEN FUNDS-TOTAL SUPPLIES & MATERIALS EXPENDITURES,ALL FUNDS-TOTAL SUPPLIES & MATERIALS EXPENDITURES,GEN FUNDS-TOTAL OTHER OPERATING EXPENDITURES,ALL FUNDS-TOTAL OTHER OPERATING EXPENDITURES,GEN FUNDS-TOTAL OPERATING EXPENDITURES BY OBJ,ALL FUNDS-TOTAL OPERATING EXPENDITURES BY OBJ,GEN FUNDS-TOTAL DEBT SERVICE EXPEND BY OBJ,ALL FUNDS-TOTAL DEBT SERVICE EXPEND BY OBJ,GEN FUNDS-TOTAL CAPITAL OUTLAY EXPEND BY OBJ,ALL FUNDS-TOTAL CAPITAL OUTLAY EXPEND BY OBJ,GEN FUNDS-TOTAL NON-OPER EXPENDITURES BY OBJ,ALL FUNDS-TOTAL NON-OPER EXPENDITURES BY OBJ,GEN FUNDS-TOTAL NON-OPER AND OPER OEXPENDITURES BY OBJ,ALL FUNDS-TOTAL NON-OPER AND OPER OEXPENDITURES BY OBJ,"GEN FUNDS-INSTRUCTION + TRANSFER EXPEND-FCT11,95","ALL FUNDS-INSTRUCTION + TRANSFER EXPEND-FCT11,95","GEN FUNDS-INSTRUC RESOURCE MEDIA SERVICE EXP, FCT12","ALL FUNDS-INSTRUC RESOURCE MEDIA SERVICE, FCT12","GEN FUNDS-CURRICULUM/STAFF DEVELOPMENT EXP, FCT13","ALL FUNDS-CURRICULUM/STAFF DEVELOPMENT EXP, FCT13","GEN FUNDS-INSTRUC LEADERSHIP EXPEND, FCT21","ALL FUNDS-INSTRUC LEADERSHIP EXPEND, FCT21","GEN FUNDS-CAMPUS ADMINISTRATION EXPEND, FCT23","ALL FUNDS-CAMPUS ADMINISTRATION EXPEND, FCT23","GEN FUNDS-GUIDANCE & COUNSELING SERVICES EXP, FCT31","ALL FUNDS-GUIDANCE 7 COUNSELING SERVICES EXP, FCT31","GEN FUNDS-SOCIAL WORK SERVICES EXP, FCT32","ALL FUNDS-SOCIAL WORK SERVICES EXP, FCT32","GEN FUNDS-HEALTH SERVICES EXP, FCT33","ALL FUNDS-HEALTH SERVICES EXP, FCT33","GEN FUNDS-TRANSPORTATION EXPENDITURES, FCT34","ALL FUNDS-TRANSPORTATION EXPENDITURES, FCT34","GEN FUNDS-FOOD SERVICE EXPENDITURES, FCT35","ALL FUNDS-FOOD SERVICE EXPENDITURES, FCT35","GEN FUNDS-EXTRACURRICULAR EXPENDITURES, FCT36","ALL FUNDS-EXTRACURRICULAR EXPENDITURES,FCT36","GEN FUNDS-GENERAL ADMINISTRAT EXPEND-FCT41,80,92","ALL FUNDS-GENERAL ADMINISTRAT EXPEND-FCT41,80,92","GEN FUNDS-PLANT MAINTENANCE/OPERA EXPEND, FCT51","ALL FUNDS-PLANT MAINTENANCE/OPERA EXPEND, FCT51","GEN FUNDS-SECURITY/MONITORING SERVICE EXPEND, FCT52","ALL FUNDS-SECURITY/MONITORING SERVICE EXPEND, FCT5","GEN FUNDS-DATA PROCESSING SERVICES EXPEND, FCT53","ALL FUNDS-DATA PROCESSING SERVICES, FCT53","GEN FUNDS-COMMUNITY SERVICES, FCT61","ALL FUNDS-COMMUNITY SERVICES, FCT61",GEN FUNDS-TOTAL OPERATE EXPEND BY FUNCTION,ALL FUNDS-TOTAL OPERATE EXPEND BY FUNCTION,GEN FUNDS-NON-OPER EXP BY FUNCTION(1X-9X)(65XX),ALL FUNDS-NON-OPER EXP BY FUNCTION(1X-9X)(65XX),GEN 

In [14]:
peims_df.head().T

,0,1,2,3,4
DISTRICT NUMBER,001902,001902,001902,001902,001902
DISTRICT NAME,CAYUGA ISD,CAYUGA ISD,CAYUGA ISD,CAYUGA ISD,CAYUGA ISD
YEAR,2007,2008,2009,2010,2011
GEN FUNDS-LOCAL TAX REVENUE FROM M&O,4122552,3641351,3319164,3222688,3152618
ALL FUNDS-LLOCAL TAX REVENUE FROM M&O,4122552,3641351,3319164,3222688,3152618
GEN FUNDS-STATE REVENUE,1114179,1845232,1950169,2302369,2439570
ALL FUNDS-STATE REVENUE,1176283,2350131,2473373,2917268,3091438
GEN FUNDS-FEDERAL REVENUE,0,0,0,0,0
ALL FUNDS-FEDERAL REVENUE,250168,1852221,1714988,2851372,2687377
GEN FUNDS-OTHER LOCAL & INTERMEDIATE REVENUE,215239,183293,123568,161664,134887


In [15]:
peims_df.columns

Index(['DISTRICT NUMBER', 'DISTRICT NAME', 'YEAR',
       'GEN FUNDS-LOCAL TAX REVENUE FROM M&O',
       'ALL FUNDS-LLOCAL TAX REVENUE FROM M&O', 'GEN FUNDS-STATE REVENUE',
       'ALL FUNDS-STATE REVENUE', 'GEN FUNDS-FEDERAL REVENUE',
       'ALL FUNDS-FEDERAL REVENUE',
       'GEN FUNDS-OTHER LOCAL & INTERMEDIATE REVENUE',
       ...
       'ALL FUNDS-TOTAL OTHER USES', 'GEN FUNDS-INTERGOVERN CHARGES EXPEND',
       'ALL FUNDS-INTERGOVERN CHARGES EXPEND',
       'GEN FUNDS-DEBT SERVICE (OBJECT 6500) FOR TD',
       'ALL FUNDS-DEBT SERVICE (OBJECT 6500) FOR TD',
       'GEN FUNDS-CAPITAL PROJECTS(OBJECT 6600) FOR TD',
       'ALL FUNDS-CAPITAL PROJECTS(OBJECT 6600) FOR TD',
       'GEN FUNDS-TOTAL DISBURSEMENTS', 'ALL FUNDS-TOTAL DISBURSEMENTS',
       'FALL SURVEY ENROLLMENT'],
      dtype='object', length=150)

In [16]:
# Remove all columns between column name 'B' to 'D'
peims_expenditures_etc_df = peims_df.drop(peims_df.loc[:, 'GEN FUNDS-LOCAL TAX REVENUE FROM M&O':'ALL FUNDS-TOTAL OPERATING, OTR, DEBT SERV FIN, AND TRS EST REVEN'].columns, axis=1)

In [17]:
peims_expenditures_etc_df.head().T

,0,1,2,3,4
DISTRICT NUMBER,001902,001902,001902,001902,001902
DISTRICT NAME,CAYUGA ISD,CAYUGA ISD,CAYUGA ISD,CAYUGA ISD,CAYUGA ISD
YEAR,2007,2008,2009,2010,2011
GEN FUNDS-TOTAL PAYROLL EXPENDITURES,3405211,3729195,3952356,4177424,4102439
ALL FUNDS-TOTAL PAYROLL EXPENDITURES,3611026,5509091,5716404,6152112,6134967
GEN FUNDS-TOTAL PROFESSIONAL & CONTRACTED SERVICES EXPENDITURES,444063,463617,472396,277583,283364
ALL FUNDS-TOTAL PROFESSIONAL & CONTRACTED SERVICES EXPENDITURES,456042,934387,941931,964779,984742
GEN FUNDS-TOTAL SUPPLIES & MATERIALS EXPENDITURES,287153,323340,341296,391572,311149
ALL FUNDS-TOTAL SUPPLIES & MATERIALS EXPENDITURES,492377,752505,754588,1117922,973794
GEN FUNDS-TOTAL OTHER OPERATING EXPENDITURES,128670,137667,165611,170952,154614


In [18]:
peims_expenditures_etc_df.to_csv('../data/inter/02 - peims_expenditures_etc_df.csv', index=False)